In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import geopy as gp
import matplotlib
import shapely.geometry
import shapely.ops
import time
import pyproj # Used to convert coordinate projections
import re # Functions for regular expressions

# Process Roadbed Data

In [219]:
# File name
shape = "NYC Street Centerline (CSCL)/geo_export_15a738be-7dbc-4a0f-826d-9bd8517b040a.shp"

# Read in the shape file
shapeGDF = gpd.read_file(shape)

In [221]:
# Create a list of streets to limit the data set - the shapefile is large and any row operation is time consuming
CrownHtsSts = ('ATLANTIC AV', 'PACIFIC ST', 'DEAN ST', 'BERGEN ST', 'ST MARKS AVE', 'ALBANY AVE', 'REVERE PL', 'KINGSTON AVE', 'BROOKLYN AVE', 'NEW YORK AVE', 'NOSTRAND AVE')

# Limit the data set to brooklyn and the streets in the list above
chStsGDF = shapeGDF[(shapeGDF.borocode == "3") & (shapeGDF.st_label.isin(CrownHtsSts))]
chStsGDF.full_stree.unique()

array(['PACIFIC ST', 'DEAN ST', 'ST MARKS AVE', 'NEW YORK AVE',
       'NOSTRAND AVE', 'BERGEN ST', 'ALBANY AVE', 'KINGSTON AVE',
       'BROOKLYN AVE', 'REVERE PL'], dtype=object)

In [256]:
# Reverse the coordinates to match longitude and latitude and signs GeoDataFrame
chStsGDF['geometry'] = chStsGDF['geometry'].map(lambda linestring: shapely.ops.transform(lambda x, y: (y, x), linestring))

# Process Street Sign Data

In [34]:
# File names
signsfil = "Street_Sign_Work_Orders.csv"

# Read in the signs file
signsDF = pd.read_csv(signsfil, sep=',', header=0)

In [224]:
# Create a list of street names to limit the data frame to
CrownHtsSts = ('ATLANTIC AVENUE', 'PACIFIC STREET', 'DEAN STREET', 'BERGEN STREET', 'ST MARKS AVENUE', 'ALBANY AVENUE', 'REVERE PLACE', 'KINGSTON AVENUE', 'BROOKLYN AVENUE', 'NEW YORK AVENUE', 'NOSTRAND AVENUE')

# Limit the data set to Brooklyn, the selected streets, current sign and signs with sanitation parking restrictions
chSignsDF = signsDF[(signsDF.borough == "Brooklyn") & (signsDF.on_street.isin(CrownHtsSts)) & ((signsDF.from_street.isin(CrownHtsSts)) | (signsDF.from_street.isin(CrownHtsSts))) & (signsDF.record_type == "Current") & (signsDF.sign_description.str.contains("SANITATION BROOM SYMBOL"))]

In [36]:
# Check column types
chSignsDF.dtypes

order_number                   object
record_type                    object
order_type                     object
borough                        object
on_street                      object
on_street_suffix               object
from_street                    object
from_street_suffix             object
to_street                      object
to_street_suffix               object
side_of_street                 object
order_completed_on_date        object
sign_code                      object
sign_description               object
sign_size                      object
sign_design_voided_on_date     object
sign_location                  object
distance_from_intersection      int64
arrow_direction                object
facing_direction               object
sheeting_type                  object
support                        object
sign_notes                     object
sign_x_coord                  float64
sign_y_coord                  float64
dtype: object

In [226]:
# Check the relationship between on_street and side_of_street
pd.crosstab(chSignsDF.on_street, chSignsDF.side_of_street)
# All streets take non-zero values for E W or N S - as expected

side_of_street,E,N,S,W
on_street,,,,
ALBANY AVENUE,15,0,0,10
BERGEN STREET,0,19,21,0
BROOKLYN AVENUE,13,0,0,11
DEAN STREET,0,28,25,0
KINGSTON AVENUE,11,0,0,14
NEW YORK AVENUE,10,0,0,12
NOSTRAND AVENUE,2,0,0,5
PACIFIC STREET,0,28,26,0
REVERE PLACE,2,0,0,2


In [227]:
# Check counts of unique values of on_street, from_street and to_street
chSignsDF.groupby(['on_street', 'from_street', 'to_street']).size()

on_street        from_street      to_street      
ALBANY AVENUE    ATLANTIC AVENUE  PACIFIC STREET      3
                 BERGEN STREET    PARK PLACE          8
                                  ST MARKS AVENUE     2
                 DEAN STREET      BERGEN STREET       4
                 PACIFIC STREET   DEAN STREET         5
                 ST MARKS AVENUE  PROSPECT PLACE      3
BERGEN STREET    ALBANY AVENUE    KINGSTON AVENUE     5
                                  REVERE PLACE        4
                 KINGSTON AVENUE  BROOKLYN AVENUE    11
                 NEW YORK AVENUE  NOSTRAND AVENUE     9
                 NOSTRAND AVENUE  ROGERS AVENUE       9
                 REVERE PLACE     KINGSTON AVENUE     2
BROOKLYN AVENUE  ATLANTIC AVENUE  PACIFIC STREET      4
                 BERGEN STREET    ST MARKS AVENUE     5
                 DEAN STREET      BERGEN STREET       4
                 PACIFIC STREET   DEAN STREET         4
                 ST MARKS AVENUE  PROSPECT PLACE      

In [257]:
# Create dictionary to replace values in listed columns
replDict = {'AVENUE':'AVE', 'STREET':'ST', 'PLACE':'PL', 'BKLYN':'BROOKLYN'}
replCols = ('on_street', 'to_street', 'from_street')

# Replace street type with abbreviation in on_street, from_street and to_street to match the convention in the shapefile
for col in replCols:
    chSignsDF[f'{col}'] = chSignsDF[f'{col}'].replace(replDict, regex=True)

/var/folders/h1/jfx6ngjx2kx18xw0f878f_fr0000gn/T/ipykernel_69491/884704215.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chSignsDF[f'{col}'] = chSignsDF[f'{col}'].replace(replDict, regex=True)


In [194]:
# Create a transformer to convert form EPSG:2263 to EPSG:4326
transformer = pyproj.Transformer.from_crs("epsg:2263", "epsg:4326")

# Transform x and y coordinates and assign to columns
chSignsDF['coord_x_t'], chSignsDF['coord_y_t'] = transformer.transform(chSignsDF.sign_x_coord, chSignsDF.sign_y_coord)

# Convert from DataFrame to GeoDataFrame using EPSG:4326 as the CRS
chSignsGDF = gpd.GeoDataFrame(chSignsDF, geometry=gpd.points_from_xy(chSignsDF.coord_x_t, chSignsDF.coord_y_t), crs = 4326)

0      POINT (40.67783 -73.95254)
1      POINT (40.67767 -73.95014)
2      POINT (40.67770 -73.95060)
3      POINT (40.67778 -73.95187)
4      POINT (40.67780 -73.95219)
                  ...            
295    POINT (40.67669 -73.94986)
296    POINT (40.67391 -73.93896)
297    POINT (40.67697 -73.94711)
298    POINT (40.67422 -73.93893)
299    POINT (40.67449 -73.93890)
Name: geometry, Length: 300, dtype: geometry

In [195]:
# Create dictionary to detect days of the week in the sign description
days = {'monday':'MONDAY', 'tuesday':'TUESDAY', 'wednesday':'WEDNESDAY', 'thursday':'THURSDAY', 'friday':'FRIDAY', 'saturday':'SATURDAY', 'sunday':'SUNDAY'}

# Create dummy variables to indicate what days the parking restrictions
for (key,value) in days.items():
    # Indicate whether the day appears on the sign
    chSignsGDF[f'{key}'] = np.where(chSignsGDF.sign_description.str.contains(value), 1, 0)
    # Reverse the dummy variable values when the word "EXCEPT" appears
    chSignsGDF[f'{key}'] = np.where(chSignsGDF.sign_description.str.contains("EXCEPT"), ((chSignsGDF[f'{key}']-1) ** 2), chSignsGDF[f'{key}'])

In [196]:
# Create variable for start time of restrictions
chSignsGDF['time_start'] = chSignsGDF.sign_description.str.extract(r'(\s.{1,5}\wM-)').replace('-|\s','', regex = True)
chSignsGDF['time_start'] = (np.where(chSignsGDF.time_start.str.contains(':'), 
                                        chSignsGDF.time_start, 
                                        np.where(chSignsGDF.time_start.str.find('M')-1 == 1, 
                                                 (chSignsGDF.time_start.str[:1] + ":00" + chSignsGDF.time_start.str[-2:]),
                                                 (chSignsGDF.time_start.str[:2] + ":00" + chSignsGDF.time_start.str[-2:]))))
chSignsGDF['time_start'] = pd.to_datetime(chSignsGDF.time_start, format='%I:%M%p').dt.time

# Create variable for end time of restrictions
chSignsGDF['time_end'] = chSignsGDF.sign_description.str.extract(r'(-.{1,5}\wM\s)').replace('-|\s','', regex = True)
chSignsGDF['time_end'] = (np.where(chSignsGDF.time_end.str.contains(':'), 
                                        chSignsGDF.time_end, 
                                        np.where(chSignsGDF.time_end.str.find('M')-1 == 1,  
                                                 (chSignsGDF.time_end.str[:1] + ":00" + chSignsGDF.time_end.str[-2:]),
                                                 (chSignsGDF.time_end.str[:2] + ":00" + chSignsGDF.time_end.str[-2:]))))
chSignsGDF['time_end'] = pd.to_datetime(chSignsGDF.time_end, format='%I:%M%p').dt.time

# Link the Roadbed and Street Sign Datasets

In [251]:
# Project the datasets to a distance CRS
chSignsGDF = chSignsGDF.to_crs(3857)
chStsGDF = chStsGDF.to_crs(3857)

# Create shards
shards = {k:d for k, d in chStsGDF.groupby('st_label')}

# Get the nearest roadbed to each sign where the street name is the same
chStSignsGDF = chSignsGDF.groupby("on_street", group_keys=True).apply(lambda d: gpd.sjoin_nearest(d, shards[d["on_street"].values[0]], how = 'inner'))

In [254]:
# Join the sjoined GeoDataFrame with the roadbeds geometry (geometry from right GDF is dropped in sjoin)
chStSignsGDF = pd.merge(chStSignsGDF, chStsGDF[['physicalid','geometry']], how = 'outer', on='physicalid')

,index,order_number,record_type,order_type,borough,on_street_left,on_street_suffix,from_street,from_street_suffix,to_street,...,snow_pri,st_label,st_name,st_width,status,to_lvl_co,trafdir,on_street_right,geometry_y,geometry
0,3758592.0,S-01496406,Current,S-,Brooklyn,ALBANY AVE,NaN,ATLANTIC AVE,NaN,PACIFIC ST,...,C,ALBANY AVE,ALBANY,32.0,2,13.0,FT,ALBANY AVE,"LINESTRING (4528228.634 -12490807.531, 4528148...","LINESTRING (4528228.634 -12490807.531, 4528148..."
1,11266862.0,S-01340049,Current,S-,Brooklyn,ALBANY AVE,NaN,ATLANTIC AVE,NaN,PACIFIC ST,...,C,ALBANY AVE,ALBANY,32.0,2,13.0,FT,ALBANY AVE,"LINESTRING (4528228.634 -12490807.531, 4528148...","LINESTRING (4528228.634 -12490807.531, 4528148..."
2,11766806.0,S-01340049,Current,S-,Brooklyn,ALBANY AVE,NaN,ATLANTIC AVE,NaN,PACIFIC ST,...,C,ALBANY AVE,ALBANY,32.0,2,13.0,FT,ALBANY AVE,"LINESTRING (4528228.634 -12490807.531, 4528148...","LINESTRING (4528228.634 -12490807.531, 4528148..."
3,3942230.0,S-590419,Current,S-,Brooklyn,ALBANY AVE,NaN,PACIFIC ST,NaN,DEAN ST,...,C,ALBANY AVE,ALBANY,32.0,2,13.0,FT,ALBANY AVE,"LINESTRING (4528148.584 -12490832.484, 4528061...","LINESTRING (4528148.584 -12490832.484, 4528061..."
4,9426692.0,S-01497062,Current,S-,Brooklyn,ALBANY AVE,NaN,PACIFIC ST,NaN,DEAN ST,...,C,ALBANY AVE,ALBANY,32.0,2,13.0,FT,ALBANY AVE,"LINESTRING (4528148.584 -12490832.484, 4528061...","LINESTRING (4528148.584 -12490832.484, 4528061..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (4523190.239 -12490165.313, 4523173...","LINESTRING (4523190.239 -12490165.313, 4523173..."
717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (4525343.699 -12494084.976, 4525278...","LINESTRING (4525343.699 -12494084.976, 4525278..."
718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (4523045.473 -12492030.282, 4523024...","LINESTRING (4523045.473 -12492030.282, 4523024..."
719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (4523086.882 -12494393.531, 4522994...","LINESTRING (4523086.882 -12494393.531, 4522994..."


In [255]:
chStSignsGDF.to_csv('chStSigns_clean.csv')